In [1]:
import torch

# Задание 1

Напишите функцию `function01`. Она должна иметь следующую сигнатуру:

`def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:`

Если `count_over` равен `'columns'`, верните среднее тензора по колонкам. Если равен `'rows'`, то верните среднее по рядам. Гарантируется, что тензор будет матрицей (то есть будет иметь размерность 2).

In [11]:
def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:
    result = torch.mean(tensor, dim=1) if count_over == 'rows' else torch.mean(tensor, dim=0)
    return result

In [12]:
test = torch.rand((2, 5))

print(test)

print(function01(test, 'rows'))
print(function01(test, 'columns'))

tensor([[0.9731, 0.2700, 0.9100, 0.2467, 0.2202],
        [0.4365, 0.3612, 0.1924, 0.5175, 0.9052]])
tensor([0.5240, 0.4826])
tensor([0.7048, 0.3156, 0.5512, 0.3821, 0.5627])



# Задание 2

Напишите функцию `function02`. Функции на вход должен приходить датасет — тензор-матрица признаков объектов. Ваша функция должна создать тензор-вектор с весами (пусть они будут из равномерного распределения на отрезке от 0 до 1) и вернуть их для дальнейшего обучения линейной регрессии без свободного коэффициента. Сделайте эти веса типа `float32`, для них нужно будет в процессе обучения вычислять градиенты (воспользуйтесь `requires_grad`).



In [20]:
def function02(tensor: torch.Tensor) -> torch.Tensor:
    w = torch.rand(tensor.shape[1], dtype=torch.float32, requires_grad=True)
    return w

In [21]:
function02(test)

tensor([0.1685, 0.6305, 0.1092, 0.8027, 0.7927], requires_grad=True)

In [22]:
torch.matmul(test, function02(test).t())

tensor([1.6156, 1.2080], grad_fn=<MvBackward0>)

# Задание 3

Напишите функцию `function03`. Она должна принимать тензор-матрицу с объектами и тензор-вектор с правильными ответами, будем решать задачу регрессии: `def function03(x: torch.Tensor, y: torch.Tensor)`:

Создайте внутри функции веса для линейной регрессии (без свободного коэффициента), можете воспользоваться функцией из предыдущего степа. С помощью градиентного спуска подберите оптимальные веса для входных данных (используйте длину шага около `1e-2`). Верните тензор-вектор с оптимальными весами из функции. Ваши обученные веса должны давать `MSE` на обучающей выборке меньше единицы.

In [ ]:
# 5 x 4 @ 4 x 1 -> 5 x 1

In [ ]:
test.zero_

In [24]:
test.requires_grad = True
test.grad

In [5]:
def function03(x: torch.Tensor, y: torch.Tensor):
    step = 1e-2
    w = torch.rand(x.shape[1], dtype=torch.float32, requires_grad=True)
    
    while True:
        y_pred = (x @ w.t()).ravel()
        mse = torch.mean((y - y_pred) ** 2)
        mse.backward()
        
        if mse < 1:
            return w
        
        with torch.no_grad():
            w -= step * w.grad
        
        w.grad.zero_()

In [6]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

w = function03(X, Y)

In [43]:
torch.mean(((X @ w.t()) - Y) ** 2)

tensor(0.9661, grad_fn=<MeanBackward0>)

# Задание 4

Напишите функцию `function04`. Она должна принимать тензор-матрицу с объектами и тензор с правильными ответами, будем решать задачу регрессии: `def function04(x: torch.Tensor, y: torch.Tensor):`

Создайте внутри функции полносвязный слой, обучите этот полносвязный слой на входных данных с помощью градиентного спуска (используйте длину шага около 1e-2). Верните его из функции. Ваш полносвязный слой должен давать `MSE` на обучающей выборке меньше 0.3.

In [20]:
import torch.nn as nn

def function04(x: torch.Tensor, y: torch.Tensor) -> torch.nn.Linear:
    
    out_features = 1 if y.ndim == 1 else y.shape[0]
    layer = nn.Linear(in_features=x.shape[1], out_features=out_features)
    step = 1e-2
    while True:
        y_pred = layer(x).ravel()
        mse = torch.mean((y_pred - y) ** 2)
        mse.backward()
        
        if mse < 0.3:
            return layer
        
        with torch.no_grad():
            layer.weight -= step * layer.weight.grad
            layer.bias -= step * layer.bias.grad
            
        layer.zero_grad()

In [21]:
layer = function04(X, Y)

In [22]:
layer = nn.Linear(in_features=X.shape[1], out_features=Y.shape[0])

In [26]:
Y

tensor([ 2.1721,  2.9376, -2.6368, -0.6423,  1.1223,  4.3183,  1.7834,  3.8030,
        -1.5654,  3.4786, -4.1216, -3.1978, -3.5100,  1.4994, -3.1281,  0.4475,
        -0.0897,  2.2667, -3.5698, -3.3759, -1.0608,  1.0593, -3.2828, -3.8367,
        -0.7850,  2.7310,  0.1547, -2.4260,  0.7030,  4.3691, -1.5849,  1.3319,
         2.7297,  0.8296, -0.9350, -2.4084, -0.6570, -0.4758, -0.4140, -3.9284,
        -3.2118,  1.9100, -2.4981, -4.0884, -4.8914,  2.2569, -1.0902, -0.6397,
         0.6339,  3.0710, -3.2772, -2.9522, -2.6351,  3.9192, -0.5444, -4.0229,
         3.9050,  3.5146, -0.0304, -3.7932, -1.7378, -3.8651, -3.7100,  4.9246,
        -0.9597, -1.5853, -2.4326, -4.2317, -2.7564,  3.3147,  1.9527,  4.3455,
         1.0749,  0.4111, -1.3890,  1.3347, -5.0745,  0.5908,  4.8613,  2.8471,
         1.7008, -2.8906, -2.5066,  2.3527, -1.6173,  3.8387, -3.5366,  3.2961,
        -3.7041, -1.7643,  0.5645, -0.6159,  4.4446,  1.1422, -4.4085, -2.3697,
        -0.5402, -4.0997, -3.1368, -2.76

In [28]:
layer(X).ravel().shape

torch.Size([90000])